# Import Required Libraries

In [1]:
import keras
import pandas as pd
import numpy as np
from collections import Counter
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib
from keras.datasets import mnist
from pandas import DataFrame, read_csv
import random
import os
import math
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

os.chdir('C:/Users/Chris/Documents/Neural Networks/Final Project')

USE_GPU = True
if USE_GPU:
    print(tf.config.list_physical_devices('GPU'))


[]


In [2]:
data_df = read_csv('bank_categorical_translate.csv')

In [3]:
training_data_df = data_df.sample(math.ceil(len(data_df.index)*0.8))
df_all = data_df.merge(training_data_df.drop_duplicates(), on=list(data_df.columns), 
                   how='left', indicator=True)
df_all = df_all.loc[df_all['_merge'] == 'left_only']
validation_data_df = df_all.drop('_merge', axis=1)
training_target_df = training_data_df.pop('target')
validation_target_df = validation_data_df.pop('target')

# Construct the Network

In [4]:
# assemble the fully connected network
number_of_columns = training_data_df.shape[1]
model = Sequential()

model.add(keras.layers.Dense(64, activation="LeakyReLU", input_shape=(number_of_columns,)))

# add the fully connected layers in a funnel shape. Try it out!
for layer_size in [64, 32, 16, 8, 4,2]:
    model.add(keras.layers.Dense(layer_size, activation='LeakyReLU'))
    # model.add(keras.layers.Dropout(.05, noise_shape=None, seed=None))
    # model.add(keras.layers.BatchNormalization())

model.add(Flatten())
model.add(keras.layers.Dense(1, activation = 'sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                3264      
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 16)                528       
                                                                 
 dense_4 (Dense)             (None, 8)                 136       
                                                                 
 dense_5 (Dense)             (None, 4)                 36        
                                                                 
 dense_6 (Dense)             (None, 2)                 1

# Train the Neural Network

In [5]:
BATCH_SIZE = 2

model.compile(optimizer='adam', loss='binary_crossentropy', 
              metrics=['accuracy'])

print("Starting Training.")
history = model.fit(training_data_df, training_target_df, validation_data=(validation_data_df, validation_target_df),
                    batch_size=BATCH_SIZE, epochs=1)
print("Training is complete.")

Starting Training.
16476/16476 [==============================] - 30s 2ms/step - loss: 0.2892 - accuracy: 0.8968 - val_loss: 0.2957 - val_accuracy: 0.8906
Training is complete.


In [6]:
print("Starting Testing.")
scores = model.evaluate(validation_data_df, validation_target_df)
print("Testing is Complete.")

Starting Testing.
241/241 [==============================] - 0s 1ms/step - loss: 0.2957 - accuracy: 0.8906
Testing is Complete.


In [7]:
print("test loss, test accuracy:", scores)

test loss, test accuracy: [0.2956677973270416, 0.8906229734420776]


In [8]:
results=model.predict(validation_data_df)

241/241 [==============================] - 0s 1ms/step


In [9]:
results

array([[0.0372556 ],
       [0.03988985],
       [0.02001224],
       ...,
       [0.5780645 ],
       [0.61065495],
       [0.8761622 ]], dtype=float32)

In [10]:
flatlist=[]
for sublist in results:
    for element in sublist:
        flatlist.append(element)
results = flatlist

In [11]:
Counter(validation_target_df)

Counter({0: 6784, 1: 905})

In [12]:
validation_target_df = pd.DataFrame(validation_target_df)

In [13]:
validation_target_df['results'] = results

In [14]:
validation_target_df['Model_Results'] = np.where((validation_target_df['results'] > 0.5) & (validation_target_df['target'] == 1), 'True Positive', 
                                                    np.where((validation_target_df['results'] > 0.5) & 
                                                     (validation_target_df['target'] != 1), "False Positive", 
                                                    np.where((validation_target_df['results'] <= 0.5) & 
                                                     (validation_target_df['target'] == 1), "False Negative", "True Negative")))

In [15]:
print(validation_target_df)

       target   results   Model_Results
13          0  0.037256   True Negative
24          0  0.039890   True Negative
29          0  0.020012   True Negative
33          0  0.036234   True Negative
34          0  0.040967   True Negative
...       ...       ...             ...
41157       0  0.164432   True Negative
41164       1  0.893861   True Positive
41172       1  0.578065   True Positive
41177       0  0.610655  False Positive
41178       1  0.876162   True Positive

[7689 rows x 3 columns]


In [16]:
TN = validation_target_df['Model_Results'].value_counts()['True Negative']
FN = validation_target_df['Model_Results'].value_counts()['False Negative']
TP = validation_target_df['Model_Results'].value_counts()['True Positive']
FP = validation_target_df['Model_Results'].value_counts()['False Positive']

In [17]:
print("True Negatives, False Negatives, True Positives, False Positives:", [TN,FN,TP,FP])

True Negatives, False Negatives, True Positives, False Positives: [6600, 657, 248, 184]


In [18]:
print("Accuracy , Precision, Recall", [(TP + TN)/(TN + FN + TP + FP)*100, 100*(TP/(TP + FP)), 100*(TP/(TP +FN))])

Accuracy , Precision, Recall [89.06229678761868, 57.407407407407405, 27.40331491712707]
